In [ ]:
!pip install object-detection-fastai
!pip install git+https://github.com/ildoonet/pytorch-randaugment

  Created wheel for object-detection-fastai: filename=object_detection_fastai-0.0.6-cp36-none-any.whl size=31580 sha256=a9718db25259d8a587802a26e8515332f4e49231612fa1caac9af9efbb96d200
  Stored in directory: /root/.cache/pip/wheels/4c/79/9c/7299e94e248a7102202b922f02da86502f2e6e35dbe07706c8
Successfully built object-detection-fastai
  Cloning https://github.com/ildoonet/pytorch-randaugment to /tmp/pip-req-build-rs5e019m
  Running command git clone -q https://github.com/ildoonet/pytorch-randaugment /tmp/pip-req-build-rs5e019m
  Created wheel for RandAugment: filename=RandAugment-0.1-cp36-none-any.whl size=24211 sha256=8be83800037732e82f426d06e31b87f557de35be426e1e03ef874d3bb4b35909
  Stored in directory: /tmp/pip-ephem-wheel-cache-k1xx88ft/wheels/0d/0e/e9/f8b70c1e233491338d524d867a7e959d10bb14a16bd5379b09
Successfully built RandAugment


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
import torch
torch.cuda.get_device_name()

'Tesla P4'

In [ ]:
%cd /content/drive/My Drive/BrainHack/YOLO/

from __future__ import division

from models import *
from utils.logger import *
from utils.utils import *
from utils.datasets import *
from utils.parse_config import *
from test import evaluate

import os
import sys
import time
import datetime
import argparse

import torch
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision import transforms
from torch.autograd import Variable
import torch.optim as optim

if __name__ == "__main__":
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # os.makedirs("output", exist_ok=True)
    # os.makedirs("checkpoints", exist_ok=True)

    # Get data configuration
    data_config = parse_data_config("config/coco.data")
    train_path = data_config["train"]
    valid_path = data_config["valid"]
    class_names = load_classes(data_config["names"])

    EPOCH = 1
    BATCH_SIZE = 2 # 32 might be too big for Colab to handle. 16 too.
    GRAD_ACCUMULATE = 2
    MODEL_DEF = "config/yolov3.cfg"
    PRETRAINED_WEIGHTS = "/content/drive/My Drive/BrainHack/yolov3.weights"
    N_CPU = 8
    IMG_SIZE = 416
    checkpoint_interval = 1
    evaluation_interval = 1
    compute_mAP = True
    multiscale_training = True

    # Initiate model
    model = Darknet(MODEL_DEF).to(device)
    model.apply(weights_init_normal)

    # If specified we start from checkpoint
    if PRETRAINED_WEIGHTS:
        if PRETRAINED_WEIGHTS.endswith(".pth"):
            model.load_state_dict(torch.load(PRETRAINED_WEIGHTS))
        else:
            model.load_darknet_weights(PRETRAINED_WEIGHTS)

    # Get dataloader
    dataset = ListDataset(train_path, augment=True, multiscale=multiscale_training)
    dataloader = torch.utils.data.DataLoader(
        dataset,
        batch_size=BATCH_SIZE,
        shuffle=True,
        num_workers=N_CPU,
        pin_memory=True,
        collate_fn=dataset.collate_fn,
    )

    optimizer = torch.optim.Adam(model.parameters())
    # scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=0.03, steps_per_epoch=len(dataloader), epochs=10)

    metrics = [
        "grid_size",
        "loss",
        "x",
        "y",
        "w",
        "h",
        "conf",
        "cls",
        "cls_acc",
        "recall50",
        "recall75",
        "precision",
        "conf_obj",
        "conf_noobj",
    ]
  # /content/drive/My Drive/BrainHack/Data/train/train/__image_na

    for epoch in range(EPOCH):
        model.train()
        start_time = time.time()
        for batch_i, (_, imgs, targets) in enumerate(dataloader):
            batches_done = len(dataloader) * epoch + batch_i

            imgs = Variable(imgs.to(device))
            targets = Variable(targets.to(device), requires_grad=False)

            loss, outputs = model(imgs, targets)
            loss.backward()

            if batches_done % GRAD_ACCUMULATE:
                # Accumulates gradient before each step
                optimizer.step()
                # scheduler.step()
                optimizer.zero_grad()

            # ----------------
            #   Log progress
            # ----------------

            log_str = "\n---- [Epoch %d/%d, Batch %d/%d] ----\n" % (epoch, EPOCH, batch_i, len(dataloader))

            metric_table = [["Metrics", *[f"YOLO Layer {i}" for i in range(len(model.yolo_layers))]]]

            # Log metrics at each YOLO layer
            for i, metric in enumerate(metrics):
                formats = {m: "%.6f" for m in metrics}
                formats["grid_size"] = "%2d"
                formats["cls_acc"] = "%.2f%%"
                row_metrics = [formats[metric] % yolo.metrics.get(metric, 0) for yolo in model.yolo_layers]
                metric_table += [[metric, *row_metrics]]

            # Determine approximate time left for epoch
            epoch_batches_left = len(dataloader) - (batch_i + 1)
            time_left = datetime.timedelta(seconds=epoch_batches_left * (time.time() - start_time) / (batch_i + 1))

            model.seen += imgs.size(0)

        if epoch % evaluation_interval == 0:
            print("\n---- Evaluating Model ----")
            # Evaluate the model on the validation set
            precision, recall, AP, f1, ap_class = evaluate(
                model,
                path=valid_path,
                iou_thres=0.5,
                conf_thres=0.5,
                nms_thres=0.5,
                img_size=IMG_SIZE,
                batch_size=BATCH_SIZE,
            )
            evaluation_metrics = [
                ("val_precision", precision.mean()),
                ("val_recall", recall.mean()),
                ("val_mAP", AP.mean()),
                ("val_f1", f1.mean()),
            ]

            # Print class APs and mAP
            ap_table = [["Index", "Class name", "AP"]]
            for i, c in enumerate(ap_class):
                ap_table += [[c, class_names[c], "%.5f" % AP[i]]]
            print(f"---- mAP {AP.mean()}")

        if epoch % checkpoint_interval == 0:
            torch.save(model.state_dict(), f"checkpoints/yolov3_ckpt_%d.pth" % epoch)


/content/drive/.shortcut-targets-by-id/16JDsNEOXVr1AV5ftUzRu4LTl3Mla2W8b/BrainHack/YOLO


/usr/local/lib/python3.6/dist-packages/PIL/Image.py:2766: DecompressionBombWarning: Image size (123871510 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning,
Detecting objects:   0%|          | 0/737 [00:00<?, ?it/s]


---- Evaluating Model ----


Detecting objects:   0%|          | 0/737 [00:02<?, ?it/s]


---- mAP 0.0
